For each SELECTED result, create snippet...

# Snippet should include:
1. document title
2. 2 sentences from doc with highest COSINE SIMILARITY w/respect to q. 


Which is an array of size 5 of this [ (docID, TF-IDF) ]

Retrieve ranked results from Gerardo's ranker

In [9]:
'''
load original dataset, the doc collection 
'''

import pandas as pd

prefix = "~/Documents/Classes/CS537_IR/project1/chunks/"
sample_path = prefix + 'chunk_17.pkl' 
doc_collection = pd.read_pickle(sample_path)
DC = doc_collection.to_dict('records')
# use pandas read_csv(skiprows=FUNCTION)

In [3]:
# [(docID, TF-IDF)]
res = [(2, 48.44939633024195), (1, 14.534818899072588), (4, 9.084261811920367), (3, 0.0), (5, 0.0)]

In [8]:
for result in res:
    docId = result[0]
    weight = result[1]
    print(rank, weight)

5 48.44939633024195
5 14.534818899072588
5 9.084261811920367
5 0.0
5 0.0


In [11]:
result = DC[docId]
result

{'content': "Telephone numbers in Curaçao and the Caribbean Netherlands\n\nCountry Code: +599International Call Prefix: 00\n\nThe country code +599 was assigned to the Netherlands Antilles (dissolved in 2010), and is now in use by Curaçao and the Caribbean Netherlands (Bonaire, Sint Eustatius and Saba).\n\nAruba and Sint Maarten, also former parts of the Netherlands Antilles, discontinued using the code in 1986 and 2011 respectively. Aruba now uses country code +297, and Sint Maarten uses the North American Numbering Plan (NANP) country code +1 with area code 721.\n\nOriginally each island had an area code each with the length of the local numbering varying from island to island (Curaçao had six while Bonaire had only four), however beginning in 1999, the numbers were modified so that each local number was seven digits long. This change left the islands without any area codes, except for Curaçao, which has the area code of 9 along with the seven digit local number.\n\nOn October 2, 200

# get doc title

In [14]:
'''
cool python trick:
https://stackoverflow.com/questions/31426095/assign-multiple-values-of-a-list
'''

title, *text = result['content'].split('\n\n')
text = ' '.join(text)

# calculate cosine similarity between every 'sentence' in doc and q

## retrieve original, unprocessed doc-string

In [17]:
import nltk.data

sent_split = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = sent_split.tokenize(text)

# SAVE THIS VARIABLE FOR DISPLAY PURPOSES!
sentences[0]

'Country Code: +599International Call Prefix: 00 The country code +599 was assigned to the Netherlands Antilles (dissolved in 2010), and is now in use by Curaçao and the Caribbean Netherlands (Bonaire, Sint Eustatius and Saba).'

In [18]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import logging

nltk.download('stopwords')
stemer = PorterStemmer()
tokenizer = TweetTokenizer()

def stem(tokens=''):
    return [stemer.stem(w) for w in tokens]
def tokenize(text=''):
    return tokenizer.tokenize(text)
def remove_stopwords(tokens=[]):
    stop_words = set(stopwords.words('english'))
    return [w for w in tokens if not w in stop_words]
def remove_capitalization(tokens=[]):
    return [w.lower() for w in tokens]
def remove_punctuation(text=''):
    return re.sub(r'[^\w\s]', '', text)
def preprocess(text=''):
    # Remove punctuation from the text.
    text = remove_punctuation(text=text)
    # Tokenize
    tokens = tokenize(text=text)
    # Remove stop words
    tokens = remove_stopwords(tokens=tokens)
    # Remove capitalization
    tokens = remove_capitalization(tokens=tokens)
    # Stem terms
    tokens = stem(tokens=tokens)
    return tokens

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## pre-process ranked CR sentences same way as index / query

In [19]:
sent_tokens = [preprocess(sentences[i]) for i in range(len(sentences))]
sent_tokens[0]

['countri',
 'code',
 '599intern',
 'call',
 'prefix',
 '00',
 'the',
 'countri',
 'code',
 '599',
 'assign',
 'netherland',
 'antil',
 'dissolv',
 '2010',
 'use',
 'curaçao',
 'caribbean',
 'netherland',
 'bonair',
 'sint',
 'eustatiu',
 'saba']

## retrieve weights for terms in sentence, & terms in query

In [23]:
query = "state nation disput local employ the"
q = preprocess(query)
q

['state', 'nation', 'disput', 'local', 'employ']

In [28]:
'''
for each sentence in sent_tokens, calculate:
'''

# of term-i in query
q_weights = [1,4,5]

# weight of term-i in doc-j
d_weights = [23,2,3,5,.8]

In [29]:
import numpy as np

def cos_similarity(d_weights, q_weights):
    numerator = sum([d*q for d,q in zip(d_weights, q_weights)])
    d_norm = sum([d*d for d in d_weights])
    q_norm = sum([q*q for q in q_weights])
    denom = np.sqrt(d_norm * q_norm)
    return numerator/denom

In [30]:
'''
Given the weights of the terms in a document, 
and the weights of the terms in a query...
'''
cos_similarity(d_weights, q_weights)

0.2979179636515667